# Homework 2: Word Similarity

Student Name: Jia Shun Low

Student ID: 743436

## General info

<b>Due date</b>: Thursday, 18 Apr 2019 4pm

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 7% of mark for class (with 6% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib and Scikit-Learn. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>. 

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

## Overview

In this homework, you'll be quantifying the similarity between pairs of words of a dataset using different methods with the word co-occurrence in the Brown corpus and synset structure of WordNet. Firstly, you will preprocess the dataset to filter out the rare and ambiguous words. Secondly, you will calculate the similarity scores for pairs of words in the filtered dateset using Lin similarity, NPMI and LSA. Lastly, you will quantify how well these methods work by comparing to a human annotated gold-standard.

## 1. Preprocessing (2 marks)

<b>Instructions</b>: For this homework we will be comparing our methods against a popular dataset of word similarities called <a href="http://www.cs.technion.ac.il/~gabr/resources/data/wordsim353/">Similarity-353</a>. You need to first obtain this dataset, which is available on LMS. The file we will be using is called *set1.tab*. Make sure you save this in the same folder as the notebook.  Except for the header (which should be stripped out), the file is tab formated with the first two columns corresponding to two words, and the third column representing a human-annotated similarity between the two words. <b>You should ignore the subsequent columns</b>.

Here shows the first six lines of the file:

```
Word 1	Word 2	Human (mean)	1	2	3	4	5	6	7	8	9	10	11	12	13	
love	sex	6.77	9	6	8	8	7	8	8	4	7	2	6	7	8	
tiger	cat	7.35	9	7	8	7	8	9	8.5	5	6	9	7	5	7	
tiger	tiger	10.00	10	10	10	10	10	10	10	10	10	10	10	10	10	
book	paper	7.46	8	8	7	7	8	9	7	6	7	8	9	4	9	
computer	keyboard	7.62	8	7	9	9	8	8	7	7	6	8	10	3	9	
```
    
You should load this file into a Python dictionary (NOTE: in Python, tuples of strings, i.e. ("tiger","cat") can serve as the keys of a dictionary to map to their human-annotated similarity). This dataset contains many rare words: we need to filter this dataset in order for it to be better suited to the resources we will use in this assignment. So your first goal is to filter this dataset to generate a smaller test set where you will evaluate your word similarity methods.

The first filtering is based on document frequencies in the Brown corpus, in order to remove rare words. In this homework, we will be treating the paragraphs of the Brown corpus as our "documents". You can iterate over them by using the `paras` method of the corpus reader. You should remove tokens that are not alphabetic. Tokens should be lower-cased and lemmatized. Now calculate document frequencies for each word type, and use this to remove from your word similarity data any word pairs where at least one of the two words has a document frequency of less than 8 in this corpus.

For this part, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called *filtered_gold_standard*. You may check the section, <i>"For your testing"</i>, below for the expected *filtered_gold_standard*.

(1 mark)

In [1]:
import nltk
from nltk.corpus import brown
from nltk.corpus import wordnet

nltk.download("brown")
nltk.download("wordnet")

# filtered_gold_standard stores the word pairs and their human-annotated similarity in your filtered test set
filtered_gold_standard = {}

###
# Your answer BEGINS HERE
###

# some constants.
RARE_THRESHOLD = 8

# We need to create a dictionary of token to doc freq.

# We  create a list of sets, of lemmatized words. Each set represents a paragraph
# from brown.paras(), and contains all the words that are alphabetical in lowercase and lemmatized
# form. When we encounter a new token, we add it to our dictionary with initial value 0.
# At the end of adding each paragraph as a set, we then update the doc_freq counts in the 
# dictionary.

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
# this function performs the appropriate lemmatization of the words in brown corpus.
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

doc_freq_dict = {}
for para in brown.paras():
    
    # for every paragraph in .paras(), we have a list of sublist, 
    # each sublist being a sentence. We have to flatten the list
    # so we have a list of all words in the paragraph.
    para_set = set()
    paragraph_words = []
    for sentence in para:
        for word in sentence:
            paragraph_words.append(word) 
    
    # After flattening, we go through each word, check if .isalpha(), 
    # then we lowercase and lemmatize it.
    for word in paragraph_words:
        if word.isalpha():
            processed_word = lemmatize(word.lower())
            para_set.add(processed_word)
            
            # if after preprocessing, we have never seen the word before, we add
            # it to the frequency dictionary, with initial value 0, because all 
            # updates to the value happen at the end of the outer for loop.
            if processed_word not in doc_freq_dict:
                doc_freq_dict[processed_word] = 0
    
    # now that we have a set of preprocessed words that occured in this paragraph, 
    # we can go through all the keys we have and increase the doc_freq by 1 if 
    # the word occured.
    for key in doc_freq_dict:
        if key in para_set:
            doc_freq_dict[key] += 1


# now, we begin to populate filtered_gold_standard only with entries that pass the rarity
# test. 

# we get every line by splitting by the newline character.

filename = 'set1.tab'
s = open(filename, 'r').read()
lines = s.split('\n')

# for every line in the set1 file, we take out the first to remove header, and the 
# last because it does not contain useful information and is a by product of splitting
# by the newline character.
for line in lines[1:-1]:
    # we take the first 3 elements of a list split by tab character, 
    # to obtain word 1, word 2 to create a tuple that will be the key, and some
    # similarity which will be the value in filtered_gold_standard
    line_elements = line.split('\t')
    w1 = line_elements[0]
    w2 = line_elements[1]

    if w1 in doc_freq_dict and w2 in doc_freq_dict:
        if doc_freq_dict[w1] >= RARE_THRESHOLD and doc_freq_dict[w2] >= RARE_THRESHOLD:
            sim_score = float(line_elements[2])
            filtered_gold_standard[(w1,w2)] = sim_score          
###
# Your answer ENDS HERE
###
print(len(filtered_gold_standard))
print(filtered_gold_standard)

[nltk_data] Downloading package brown to C:\Users\Jia
[nltk_data]     Shun\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Jia
[nltk_data]     Shun\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


94
{('love', 'sex'): 6.77, ('tiger', 'cat'): 7.35, ('tiger', 'tiger'): 10.0, ('book', 'paper'): 7.46, ('plane', 'car'): 5.77, ('train', 'car'): 6.31, ('telephone', 'communication'): 7.5, ('television', 'radio'): 6.77, ('drug', 'abuse'): 6.85, ('bread', 'butter'): 6.19, ('doctor', 'nurse'): 7.0, ('professor', 'doctor'): 6.62, ('student', 'professor'): 6.81, ('smart', 'student'): 4.62, ('smart', 'stupid'): 5.81, ('company', 'stock'): 7.08, ('stock', 'market'): 8.08, ('stock', 'phone'): 1.62, ('stock', 'egg'): 1.81, ('stock', 'live'): 3.73, ('stock', 'life'): 0.92, ('book', 'library'): 7.46, ('bank', 'money'): 8.12, ('wood', 'forest'): 7.73, ('money', 'cash'): 9.08, ('king', 'queen'): 8.58, ('bishop', 'rabbi'): 6.69, ('holy', 'sex'): 1.62, ('football', 'basketball'): 6.81, ('football', 'tennis'): 6.63, ('tennis', 'racket'): 7.56, ('law', 'lawyer'): 8.38, ('movie', 'star'): 7.38, ('movie', 'critic'): 6.73, ('movie', 'theater'): 7.92, ('space', 'chemistry'): 4.88, ('alcohol', 'chemistry'): 

<b>For your testing: </b>

In [2]:
assert(len(filtered_gold_standard) > 50 and len(filtered_gold_standard) < 100)

In [3]:
assert(filtered_gold_standard[('love', 'sex')] == 6.77)

<b>Instructions</b>: Here, you apply the second filtering. The second filtering is based on words with highly ambiguous senses and involves using the NLTK interface to WordNet. Here, you should remove any words which do not have a *single primary sense*. We define single primary sense here as either a) having only one sense (i.e. only one synset), or b) where the count (as provided by the WordNet `count()` method for the lemmas associated with a synset) of the most common sense is at least 4 times larger than the next most common sense. Note that a synset can be associated with multiple lemmas. You should only consider the count of your lemma. Also, you should remove any words where the primary sense is not a noun (this information is also in the synset). Store the synset corresponding to this primary sense in a dictionary for use in the next section. Given this definition, remove the word pairs from the test set where at least one of the words does not meet the above criteria.

When you have applied the two filtering steps, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called *final_gold_standard*. You may check the section, <i>"For your testing"</i>, below for the expected *final_gold_standard*.

(1 mark)

In [4]:
# final_gold_standard stores the word pairs and their human-annotated similarity in your final filtered test set
final_gold_standard = {}

###
# Your answer BEGINS HERE
###

# we take our filtered_gold_standard, and keep the words that have a single primary sense
# defined as having only one synset, OR the count of the most common sense is at least 
# 4 times larger than the next most common sense.
# we should also remove any word where the primary sense is not a noun, however our data set
# only contains nouns.
def one_primary_sense(word):
    # get the synsets for that word.
    synsets = wordnet.synsets(word)
    primary_sense = None
    if len(synsets) == 1:
        return True

    # if we have more than one synset, we now check for the second condition, 
    # that the most common count is at least 4 times larger
    # than the second most common count. We only do this for lemmas that match our lemma exactly.
    # we first obtain all the lemman counts.
    else:
        synset_common_count = []
        for synset in synsets:
            for lemma in synset.lemmas():
                if lemma.name().lower() == word:
                    synset_common_count.append(lemma.count())
    
    # if, after getting the counts for the lemmas which match our word, we don't even have
    # 2, then the word does not have one sense.
    if len(synset_common_count) < 2:
        return False
    
    # if we have more than 2 lemma counts, then we can do the at least 4 times more comparision 
    # between most common and second most common
    synset_common_count.sort()
    most_common_count = synset_common_count[-1]
    second_most_common_count = synset_common_count[-2]
    return second_most_common_count*4 <= most_common_count 

# now we construct a new dictionary with only pairs where both words have one primary sense
for w1,w2 in filtered_gold_standard:
    if one_primary_sense(w1) and one_primary_sense(w2):
        final_gold_standard[(w1,w2)] = filtered_gold_standard[(w1,w2)]
###
# Your answer ENDS HERE
###

print(len(final_gold_standard))
print(final_gold_standard)

26
{('bread', 'butter'): 6.19, ('professor', 'doctor'): 6.62, ('student', 'professor'): 6.81, ('stock', 'egg'): 1.81, ('money', 'cash'): 9.08, ('king', 'queen'): 8.58, ('bishop', 'rabbi'): 6.69, ('football', 'basketball'): 6.81, ('football', 'tennis'): 6.63, ('alcohol', 'chemistry'): 5.54, ('baby', 'mother'): 7.85, ('car', 'automobile'): 8.94, ('journey', 'voyage'): 9.29, ('coast', 'shore'): 9.1, ('brother', 'monk'): 6.27, ('journey', 'car'): 5.85, ('coast', 'hill'): 4.38, ('forest', 'graveyard'): 1.85, ('monk', 'slave'): 0.92, ('coast', 'forest'): 3.15, ('psychology', 'doctor'): 6.42, ('psychology', 'mind'): 7.69, ('psychology', 'health'): 7.23, ('psychology', 'science'): 6.71, ('planet', 'moon'): 8.08, ('planet', 'galaxy'): 8.11}


In [5]:
assert(len(final_gold_standard) > 10 and len(final_gold_standard) < 40)

In [6]:
assert(final_gold_standard[('professor', 'doctor')] == 6.62)

## 2. Word similiarity scores with Lin similarity, NPMI and LSA (3 marks)

<b>Instructions</b>: Now you will create several dictionaries with similarity scores for pairs of words in your test set derived using the techniques discussed in class. The first of these is the Lin similarity for your word pairs using the information content of the Brown corpus, which you should calculate using the primary sense for each word derived above. You can use the built-in method included in the NLTK interface, you don't have to implement your own. 

When you're done, you should store the word pair and similarity mappings in a dictionary called *lin_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *lin_similarities*. 

(1 mark)

In [7]:
from nltk.corpus import wordnet_ic
nltk.download('wordnet_ic')

# lin_similarities stores the word pair and Lin similarity mappings
lin_similarities = {}

###
# Your answer BEGINS HERE
###

# Given a word with more than one synset, we want to get the most common synset.
def get_most_common_synset(word, synsets):
    max_count = 0
    most_common_synset = None
    for synset in synsets:
        for lemma in synset.lemmas():
            if lemma.name() == word:
                if lemma.count() >= max_count:
                    max_count = lemma.count()
                    most_common_synset = synset
    return most_common_synset

# Given a word, we want to get a synset for calculating lin similarity.
# If it just has one synset, then we take that, otherwise we have to 
# get the most common synset. We assume these words just have one sense, 
# as they have been filterd from cell above.
def get_synset_for_lin(word):
    synsets = wordnet.synsets(word, 'n')
    if len(synsets) == 1:
        return synsets[0]
    else:
        return get_most_common_synset(word, synsets)

# calculates lin_similarity between 2 words, in this notebook we only use
# brown ic, but this function works in general for all ic.
def calculate_lin_similarity(w1, w2, brown_ic):
    # first we get the synset of w1, and the synset of word 2
    synset_w1 = get_synset_for_lin(w1)
    synset_w2 = get_synset_for_lin(w2)
    return synset_w1.lin_similarity(synset_w2, brown_ic) 

brown_ic = wordnet_ic.ic('ic-brown.dat')

# construct lin similarities dicitonary
for key in final_gold_standard:
    w1,w2 = key
    lin_similarities[key] = calculate_lin_similarity(w1,w2, brown_ic)
###
# Your answer ENDS HERE
###

print(lin_similarities)

[nltk_data] Downloading package wordnet_ic to C:\Users\Jia
[nltk_data]     Shun\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


{('bread', 'butter'): 0.711420490146294, ('professor', 'doctor'): 0.7036526610448273, ('student', 'professor'): 0.26208607023317687, ('stock', 'egg'): -0.0, ('money', 'cash'): 0.7888839126424345, ('king', 'queen'): 0.25872135992145145, ('bishop', 'rabbi'): 0.6655650900427844, ('football', 'basketball'): 0.7536025025710653, ('football', 'tennis'): 0.7699955045932811, ('alcohol', 'chemistry'): 0.062235427146896456, ('baby', 'mother'): 0.6315913189894092, ('car', 'automobile'): 1.0, ('journey', 'voyage'): 0.6969176573027711, ('coast', 'shore'): 0.9632173804623256, ('brother', 'monk'): 0.24862817480738675, ('journey', 'car'): -0.0, ('coast', 'hill'): 0.5991131628821826, ('forest', 'graveyard'): -0.0, ('monk', 'slave'): 0.2543108201944307, ('coast', 'forest'): -0.0, ('psychology', 'doctor'): -0.0, ('psychology', 'mind'): 0.304017384194818, ('psychology', 'health'): 0.06004979886905243, ('psychology', 'science'): 0.8474590505736942, ('planet', 'moon'): 0.14532306834462655, ('planet', 'galaxy

<b>For your testing:</b>

In [8]:
assert(lin_similarities[('professor', 'doctor')] > 0.5 and lin_similarities[('professor', 'doctor')] < 1)

**Instructions:** Next, you will calculate Normalized PMI (NPMI) for your word pairs using word frequency derived from the Brown.

PMI is defined as:

\begin{equation*}
PMI = \log_2\left(\frac{p(x,y)}{p(x)p(y)}\right)
\end{equation*}

where

\begin{equation*}
p(x,y) = \frac{\text{Number of paragraphs with the co-occurrence of x and y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(x) = \frac{\text{Number of paragraphs with the occurrence of x}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(y) = \frac{\text{Number of paragraphs with the occurrence of y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

with the sum over $i$ ranging over all paragraphs. Note that there are other ways PMI could be formulated.

NPMI is defined as:

\begin{equation*}
NPMI = \frac{PMI}{-\log_2(p(x,y))} = \frac{\log_2(p(x)p(y))}{\log_2(p(x,y))} - 1
\end{equation*}

Thus, when there is no co-occurrence, NPMI is -1. NPMI is normalized between [-1, +1].

You should use the same set up as you did to calculate document frequency above: paragraphs as documents, lemmatized, lower-cased, and with term frequency information removed by conversion to Python sets. You need to use the basic method for calculating PMI introduced in class (and also in the reading) which is appropriate for any possible definition of co-occurrence (here, there is co-occurrence when a word pair appears in the same paragraph), but you should only calculate PMI for the words in your test set. You must avoid building the entire co-occurrence matrix, instead you should keeping track of the sums you need for the probabilities as you go along. 

When you have calculated NPMI for all the pairs, you should store the word pair and NPMI-similarity mappings in a dictionary called *NPMI_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *NPMI_similarities*. 

(1 mark)

In [9]:
# NPMI_similarities stores the word pair and NPMI similarity mappings
NPMI_similarities = {}

###
# Your answer BEGINS HERE
###
import math

# Function that takes in the brown corpus, and returns
# a list of sets, each set corresponding to the words in a paragraph.
def create_brown_list(brown):
    brown_list = []
    for para in brown.paras():
        para_set = set()
        for word in para[0]:
            if word.isalpha():
                processed_word = lemmatize(word.lower())
                para_set.add(processed_word)
        brown_list.append(para_set)
    return brown_list

# calculates numerator for the p(x) and p(y) functions in NPMI.
def calculate_p_word_numerator(word, brown_list):
    numerator = 0
    for para in brown_list:
        if word in para:
            numerator += 1
    return numerator

# calculates the numerator for p(x,y) function in NPMI
def calculate_p_cooccurrence_numerator(word1, word2, brown_list):
    numerator = 0
    for para in brown_list:
        if word1 in para and word2 in para:
            numerator += 1
    return numerator

# calcualtes the denominator for p(x) and p(x,y) functions in NPMI.
def calculate_p_denominator(brown_list):
    denominator = 0
    for para in brown_list:
        denominator += len(para)
    return denominator

# Given a list of sets, each set containing words in a para, and a word_pair, calculates 
# the NPMI_similarity
def NPMI_calculation(word_pair, brown_list, p_denominator):
    x = word_pair[0]
    y = word_pair[1]
    p_x = calculate_p_word_numerator(x, brown_list)/p_denominator
    p_y = calculate_p_word_numerator(y, brown_list)/p_denominator
    p_x_y = calculate_p_cooccurrence_numerator(x,y, brown_list)/p_denominator
    if p_x_y == 0:
        return -1
    return math.log(p_x*p_y,2)/math.log(p_x_y,2) - 1

# Using functions defined above, creates NPMI_similarities dictionary
brown_list = create_brown_list(brown)
p_denominator = calculate_p_denominator(brown_list)
for word_pair in final_gold_standard:
    NPMI_similarities[word_pair] = NPMI_calculation(word_pair, brown_list, p_denominator)
###
# Your answer ENDS HERE
###

print(NPMI_similarities)

{('bread', 'butter'): 0.675770944127283, ('professor', 'doctor'): -1, ('student', 'professor'): -1, ('stock', 'egg'): -1, ('money', 'cash'): 0.4444037486658723, ('king', 'queen'): 0.5120636847952567, ('bishop', 'rabbi'): -1, ('football', 'basketball'): 0.69758913584575, ('football', 'tennis'): -1, ('alcohol', 'chemistry'): -1, ('baby', 'mother'): 0.3924720350043247, ('car', 'automobile'): 0.4321828806781993, ('journey', 'voyage'): -1, ('coast', 'shore'): -1, ('brother', 'monk'): -1, ('journey', 'car'): -1, ('coast', 'hill'): -1, ('forest', 'graveyard'): -1, ('monk', 'slave'): -1, ('coast', 'forest'): -1, ('psychology', 'doctor'): -1, ('psychology', 'mind'): -1, ('psychology', 'health'): -1, ('psychology', 'science'): -1, ('planet', 'moon'): 0.7145517939077397, ('planet', 'galaxy'): -1}


<b>For your testing:</b>

In [10]:
assert(NPMI_similarities[('professor', 'doctor')] == -1)

**Instructions:** As PMI matrix is very sparse and can be approximated well by a dense representation via singular value decomposition (SVD), you will derive similarity scores using the Latent Semantic Analysis (LSA) method, i.e. apply SVD and truncate to get a dense vector representation of a word type and then calculate cosine similarity between the two vectors for each word pair. You can use the Distributed Semantics notebook as a starting point, but note that since you are interested here in word semantics, you will be constructing a matrix where the (non-sparse) rows correspond to words in the vocabulary, and the (sparse) columns correspond to the texts where they appear (this is the opposite of the notebook). Again, use the Brown corpus, in the same format as with PMI and document frequency. After you have a matrix in the correct format, use `truncatedSVD` in `sklearn` to produce dense vectors of length k = 500, and then use cosine similarity to produce similarities for your word pairs. 

When you are done, you should store the word pair and LSA-similarity mappings in a dictionary called *LSA_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *LSA_similarities*. 

(1 mark)

In [11]:
# LSA_similarities stores the word pair and LSA similarity mappings
LSA_similarities = {}

###
# Your answer BEGINS HERE
###
import numpy as np
from sklearn.decomposition import TruncatedSVD

# We have a dictionary of word to row, so we can use that to know which word 
# a certain row is referring to. We use this to access the appropriate vectors 
# represented by the rows, for any calculations.
word_to_row = {}
i = 0
sparse_matrix = []

# create our sparse matrix, with the rows being words, and columns being whether
# that word is in a document or not.
for word in doc_freq_dict:
    row = []
    word_to_row[word] = i
    i += 1
    # now we create that row, then append it to our sparse matrix.
    for para in brown_list:
        if word in para:
            row.append(1)
        else:
            row.append(0)
    sparse_matrix.append(row)
    
# now we convert the sparse matrix into numpy array, so we can 
# run the svd. 
sparse_matrix = np.array(sparse_matrix)

# now we transform the sparse matrix into a lower rank matrix. 
svd = TruncatedSVD(n_components=500)
matrix_lowrank = svd.fit_transform(sparse_matrix)

# for each pair, we get the vector by referring to the row of the word.
# we get the row of the word by referring to the word to row dictionary.

# gets a magnitude of a vector, abstracted here so code is more readable
# for those not familiar with random numpy functions. 
def magnitude(vec):
    return np.linalg.norm(vec)

# calculates cosine similarity
def cosine_sim(v1,v2):
    numerator = np.dot(v1,v2)
    denominator = magnitude(v1)*magnitude(v2)
    return numerator/denominator

# creates LSA_similarities dictionary, with the word vectors from our lowrank matrix
# and cosine similarities between the vectors for the scores.
for word1, word2 in final_gold_standard:
    word_vec_1 = matrix_lowrank[word_to_row[word1]]
    word_vec_2 = matrix_lowrank[word_to_row[word2]]
    LSA_similarities[(word1, word2)] = cosine_sim(word_vec_1, word_vec_2)
###
# Your answer ENDS HERE
###

print(LSA_similarities)

{('bread', 'butter'): 0.3680537092209177, ('professor', 'doctor'): 0.04771843253684325, ('student', 'professor'): -0.025129221594411423, ('stock', 'egg'): -0.009191609752586765, ('money', 'cash'): 0.21502483127265923, ('king', 'queen'): 0.11820474502167592, ('bishop', 'rabbi'): 0.04344493710754035, ('football', 'basketball'): 0.17940389201651458, ('football', 'tennis'): -0.004167895189318334, ('alcohol', 'chemistry'): -0.010279893696664613, ('baby', 'mother'): 0.11258355803110169, ('car', 'automobile'): 0.1366546736211385, ('journey', 'voyage'): 0.04695542417698878, ('coast', 'shore'): 0.11680829726377648, ('brother', 'monk'): -0.030050372956540554, ('journey', 'car'): -0.008515484195502555, ('coast', 'hill'): 0.03743261517805603, ('forest', 'graveyard'): -0.03016800991246056, ('monk', 'slave'): 0.008835472508128885, ('coast', 'forest'): 0.0022361650913561635, ('psychology', 'doctor'): -0.02608434418569705, ('psychology', 'mind'): 0.008212150787906245, ('psychology', 'health'): -0.0203

<b>For your testing:</b>

In [12]:
assert(LSA_similarities[('professor', 'doctor')] > 0 and LSA_similarities[('professor', 'doctor')] < 0.4)

## 3. Comparison with the Gold Standard (1 mark)


**Instructions:** Finally, you should compare all the similarities you've created to the gold standard you loaded and filtered in the first step. For this, you can use the Pearson correlation co-efficient (`pearsonr`), which is included in scipy (`scipy.stats`). Be careful converting your dictionaries to lists for this purpose, the data for the two datasets needs to be in the same order for correct comparison using correlation. Write a general function, then apply it to each of the similarity score dictionaries.

When you are done, you should put the result in a dictionary called *pearson_correlations* (use the keys: 'lin', 'NPMI', 'LSA'). You may check the section, <i>"For your testing"</i>, below for the expected *pearson_correlations*. 

<b>Hint:</b> All of the methods used here should be markedly above 0, but also far from 1 (perfect correlation); if you're not getting reasonable results, go back and check your code for bugs!  

(1 mark)


In [13]:
from scipy.stats import pearsonr

# pearson_correlations stores the pearson correlations with the gold standard of 'lin', 'NPMI', 'LSA'
pearson_correlations = {}

###
# Your answer BEGINS HERE
###

# to ensure the entries in each array correspond to the same word, we will iterate through
# the gold standard dictionary, get that pair, and populate all score arrays.
num_pairs = len(final_gold_standard)

lin_scores = np.zeros(num_pairs)
NPMI_scores = np.zeros(num_pairs)
LSA_scores = np.zeros(num_pairs)
final_gold_standard_scores = np.zeros(num_pairs)

i = 0
for pair in final_gold_standard:
    final_gold_standard_scores[i] = final_gold_standard[pair]
    NPMI_scores[i] = NPMI_similarities[pair]
    lin_scores[i] = lin_similarities[pair]
    LSA_scores[i] = LSA_similarities[pair]
    i += 1

# now with all our score arrays, we can calculate correlation an dput it into pearson correlations
# dictionary. We only take the 0th element of the return tuple, as we are not interested in the p-value,
# just the coefficient score.
pearson_correlations['lin'] = pearsonr(lin_scores, final_gold_standard_scores)[0]
pearson_correlations['LSA'] = pearsonr(LSA_scores, final_gold_standard_scores)[0]
pearson_correlations['NPMI'] = pearsonr(NPMI_scores, final_gold_standard_scores)[0]
###
# Your answer ENDS HERE
###

print(pearson_correlations)

{'lin': 0.48165028483571143, 'LSA': 0.39775264787480963, 'NPMI': 0.39325397087988434}


<b>For your testing:</b>

In [14]:
assert(pearson_correlations['lin'] > 0.4 and pearson_correlations['lin'] < 0.8)

## Challenge yourself: Improving the correlation (This part will NOT be marked)

You can try to derive a similarity score from word2vec vectors, using the Gensim interface, and compare it with the similarity scores you've created and the gold standard. Check the Gensim word2vec tutorial for details on the API: https://radimrehurek.com/gensim/models/word2vec.html. Again, you should use the Brown for this, but for word2vec you don't need to worry about paragraphs: feel free to train your model at the sentence level instead. Your vectors should have the same number of dimensions as LSA (500), and you need to run for 50 iterations. This may take a while (several minutes).

## A final word

Normally, we would not use a corpus as small as the Brown for the purposes of building distributional word vectors. Also, note that filtering our test set to just words we are likely to do well on would typically be considered cheating.